# Implementación del Modelo SVM 
Como segundo modelo, hemos decidido implementar el SVM con margen suave (Soft Margin)

In [1]:
import numpy as np
from itertools import product

## Algoritmo de SVM

In [ ]:
class LinearSVM_SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit_SVM(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, self.w) + self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y[idx]))
                    self.b -= self.lr * y[idx]

    def predict_SVM(self, X):
        return np.sign(np.dot(X, self.w) + self.b)


## Funciones para las métricas 


In [ ]:
def confusion_matrix_SVM(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == -1) & (y_pred == -1))
    fp = np.sum((y_true == -1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == -1))
    return np.array([[tn, fp],[fn, tp]])

def precision_SVM(y_true, y_pred):
    cm = confusion_matrix_SVM(y_true, y_pred)
    tp = cm[1,1]
    fp = cm[0,1]
    return tp / (tp + fp + 1e-9)

def recall_SVM(y_true, y_pred):
    cm = confusion_matrix_SVM(y_true, y_pred)
    tp = cm[1,1]
    fn = cm[1,0]
    return tp / (tp + fn + 1e-9)

def f1_score_SVM(y_true, y_pred):
    p = precision_SVM(y_true, y_pred)
    r = recall_SVM(y_true, y_pred)
    return 2 * (p * r) / (p + r + 1e-9)



## Función para hallar mejores Hiperparámetros

In [ ]:
def hyperparameter_search_SVM(X_train, y_train, X_val, y_val, lr_list, lambda_list, iter_list):
    best_params = None
    best_f1 = -1
    for lr, lam, it in product(lr_list, lambda_list, iter_list):
        model = LinearSVM_SVM(learning_rate=lr, lambda_param=lam, n_iters=it)
        model.fit_SVM(X_train, y_train)
        preds = model.predict_SVM(X_val)
        score = f1_score_SVM(y_val, preds)
        if score > best_f1:
            best_f1 = score
            best_params = (lr, lam, it)
    return best_params, best_f1


## Entrenamiento del Modelo
